# Maintenance du modèle de segmentation client

Dans ce notebook, on simule l'évolution du temps pour évaluer
à quelle fréquence le modèle de segmentation (KMeans k=6) doit être mis à jour
pour rester pertinent.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import adjusted_rand_score
import joblib
import matplotlib.pyplot as plt
import logging

logging.getLogger("matplotlib.font_manager").setLevel(logging.ERROR)

# Chargement des segments et du modèle
seg_k6 = pd.read_csv("olist_customers_segments_k6.csv", index_col=0)

kmeans = joblib.load("../models/kmeans_model_k6.joblib")
scaler = joblib.load("../models/scaler_features.joblib")

features = [
    "recency_days", 
    "frequency",
    "monetary",
    "avg_review_score",
    "avg_delivery_delay"
    ]

X_base = seg_k6[features].values
labels_base = seg_k6["cluster"].values

